In [14]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_parquet("../data/processed/products_clustering.parquet")



In [6]:
# def ensure_dtypes(df):
#     from pandas import dtype

#     schema = {
#         "id": dtype("int64"),
#         "subscribers_count": dtype("int64"),
#         "float_price": dtype("float64"),
#         "is_ps5": dtype("int64"),
#         "is_ps4": dtype("int64"),
#         "is_ps3": dtype("int64"),
#         "is_psv": dtype("int64"),
#         "platforms_count": dtype("int64"),
#         "has_platinum": dtype("int32"),
#     }

#     assert df.dtypes.to_dict() == schema


In [12]:
X = df.drop(columns=["id"]).to_numpy()

In [15]:
scaler = StandardScaler()
std_x = scaler.fit_transform(X)


In [21]:
pca = PCA(n_components=2)
pca_x = pca.fit_transform(std_x)

pca_df = pd.DataFrame(pca_x, columns=["PC1", "PC2"])
pca_df.head()


,PC1,PC2
0,-0.551933,1.034235
1,-1.037864,5.894371
2,-0.305457,0.292100
3,-0.452008,4.201093
4,-2.379893,7.132216


In [27]:
# pca_df.plot.scatter(x="PC1", y="PC2")

In [29]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm="auto").fit(pca_x)

distances, indices = nbrs.kneighbors(pca_x)

array([[0.     , 0.00649, 0.01297, 0.01297, 0.04436],
       [0.     , 0.05727, 0.07499, 0.15565, 0.16614],
       [0.     , 0.00649, 0.00973, 0.00973, 0.00973],
       ...,
       [0.     , 0.     , 0.00487, 0.00765, 0.01493],
       [0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.     , 0.     , 0.     , 0.     , 0.     ]])

In [30]:
indices

array([[   0,   81, 3932, 2744,  386],
       [   1, 1016,  766,    8, 1528],
       [   2, 1321, 2964, 1731, 2904],
       ...,
       [7121, 6682, 4353,   42,   49],
       [7122, 1664, 6477, 6120, 6637],
       [1614, 2013, 1732, 1612, 1725]], dtype=int64)

In [105]:
nearest = pd.DataFrame(indices)
nearest.columns = ["n" + str(x) for x in nearest.columns]

In [106]:
nearest.head()

,n0,n1,n2,n3,n4
0,0,81,3932,2744,386
1,1,1016,766,8,1528
2,2,1321,2964,1731,2904
3,3,102,160,3915,183
4,4,3591,384,3923,191


In [110]:
# Convert the indices in the dataframe to the product ids
ids_dict = df["id"].copy().to_dict()
nearest_ids = nearest.replace(ids_dict).rename({"n0": "game_id"}, axis=1)


In [111]:
nearest_ids.head()

,game_id,n1,n2,n3,n4
0,5154444,4872790,4967470,1220569,4878807
1,4928471,4086377,2746521,4672196,4346346
2,5198377,4937760,2420001,4775367,2367419
3,5048054,4713771,4713773,1527384,5145108
4,5065097,536131,3198029,2461059,5055132


In [112]:
import joblib

joblib.dump(nbrs, "../models/nearest_5_auto.sav")

['../models/nearest_5_auto.sav']

In [120]:
nearest_ids.to_parquet("../data/neighbors_5_auto.parquet")